# Using DebugResult

Here, we will show how to use `DebugResult` to debug some problems we might encounter when using our mlir-opt CLI Wrapper.

Let’s first import some necessary modules and generate an instance of our mlir-opt CLI Wrapper.

In [1]:
from mlir_graphblas import MlirOptCli
from mlir_graphblas.cli import DebugResult

cli = MlirOptCli(executable=None, options=None)

## Generate Example Input

Let's say we have a bunch of MLIR code that we're not familiar with. 

In [2]:
mlir_string = """
#trait_mul_s = {
  indexing_maps = [
    affine_map<(i) -> (i)>,
    affine_map<(i) -> (i)>,
    affine_map<(i) -> (i)>
  ],
  sparse = [
    [ "S" ],
    [ "S" ],
    [ "D" ]
  ],
  iterator_types = ["parallel"],
  doc = "Sparse Vector Multiply"
}

!SparseTensor = type !llvm.ptr<i8>
func @sparse_vector_multiply(%argA: !SparseTensor, %argB: !SparseTensor) -> tensor<8xf32> {
  %output_storage = constant dense<0.0> : tensor<8xf32>
  %arga = linalg.sparse_tensor %argA : !SparseTensor to tensor<8xf32>
  %argb = linalg.sparse_tensor %argB : !SparseTensor to tensor<8xf32>
  %0 = linalg.generic #trait_mul_s
    ins(%arga, %argb: tensor<8xf32>, tensor<8xf32>)
    outs(%output_storage: tensor<8xf32>) {
      ^bb(%a: f32, %b: f32, %x: f32):
        %0 = mulf %a, %b : f32
        linalg.yield %0 : f32
  } -> tensor<8xf32>
  return %0 : tensor<8xf32>
}
"""
mlir_bytes = mlir_string.encode()

Since we're not familiar with this code, we don't exactly know what passes are necessary or in what order they should go in.

Let's say that this is the first set of passes we try. 

In [3]:
passes = [
    "--test-sparsification=lower",
    "--linalg-bufferize",
    "--func-bufferize",
    "--tensor-bufferize",
    "--tensor-constant-bufferize",
    "--finalizing-bufferize",
    "--convert-linalg-to-loops",
    "--convert-scf-to-std",
    "--convert-std-to-llvm",
]

Let's see what results we get. 

In [4]:
result = cli.apply_passes(mlir_bytes, passes)

MlirOptError: b"<stdin>:22:8: error: failed to legalize operation 'scf.condition'"

We get an exception. 

Unfortunately, the exception message isn't very clear as it only gives us the immediate error message but doesn't inform us of the context in which it occurred, e.g. in which pass the error occurred (if any) or if any necessary passes are missing. 

Let's try to use the `debug_passes` method instead of the `apply_passes` to get more information. 

In [5]:
result = cli.debug_passes(mlir_bytes, passes)

In [6]:
result

  Error when running func-bufferize  
<stdin>:26:7: error: failed to legalize operation 'scf.condition'
      scf.condition(%16) %arg2, %arg3 : index, index
      ^
<stdin>:26:7: note: see current operation: "scf.condition"(%16, %arg2, %arg3) : (i1, index, index) -> () loc("<stdin>":26:7)


  Input to func-bufferize  
            10        20        30        40        50        60        70        80        90        
   123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123
   ---------------------------------------------------------------------------------------------
 1|module  {
 2|  func private @sparseValuesF32(!llvm.ptr<i8>) -> memref<?xf32>
 3|  func private @sparseIndices64(!llvm.ptr<i8>, index) -> memref<?xindex>
 4|  func private @sparsePointers64(!llvm.ptr<i8>, index) -> memref<?xindex>
 5|  func @sparse_vector_multiply(%arg0: !llvm.ptr<i8>, %arg1: !llvm.ptr<i8>) -> tensor<8xf32> {
 6|    %cst = constant dense<0.000000e+00> : tensor<8

This large output may seem intimidating due to it's size, but it's mostly just large since it's showing the inputs to each pass. 

This output gives us some useful information. In particular, it tells us:

  * the inputs to each pass in reverse order (the most recently run, and most likely problematic, pass is shown at the top)
  * the error occurs in the `func-bufferize` pass
  * the problem occurs at line 26 of the input to the `func-bufferize` pass

The main error message shown above is "`failed to legalize operation 'scf.condition'`". 

The [MLIR documentation on dialect conversion](https://mlir.llvm.org/docs/DialectConversion/#conversion-target) gives us some useful insight on what the issue might be:

  * If a the results of a pass contained an illegal operation, the pass did not succeed.
  * The result of the `func-bufferize` pass had `scf.condition`, an illegal operation for the `func-bufferize` pass, on line 26.

If we look at `func-bufferize`'s [MLIR documentation](https://mlir.llvm.org/docs/Passes/#-func-bufferize-bufferize-funccallreturn-ops), we'll see that it's intended to bufferize `std.func` and `std.call` operations. Note that the documentation says nothing about how the `func-bufferize` is supposed to handle any `scf` operations.

Since it's the error comes from the fact that `scf.condition` was passed into `func-bufferize` and that `scf.condition` was not handled (the documentation hints that it is not intended to be), it would make sense that we not pass MLIR code with `scf.condition` as input to the `func-bufferize` pass.

`func-bufferize`'s [MLIR documentation](https://mlir.llvm.org/docs/Passes/#-func-bufferize-bufferize-funccallreturn-ops) makes it sound like it's intended to only handle operations from the `std` dialect. Let's see if there are any passes that'll convert `scf` operations to `std` operations. Since our `MlirOptCli` instance, i.e. the variable `cli` in our Python code, uses the default executable, i.e. `mlir-opt`, we can see what passes are available by looking at `mlir-opt`'s help menu. 

We won't show the full `mlir-opt` help menu here since it's very large but will instead do a `grep` search on it to see if there's anything mentioning `std`.

In [7]:
!mlir-opt --help | grep "std"

Available Dialects: acc, affine, arm_neon, arm_sve, async, avx512, complex, gpu, linalg, llvm, llvm_arm_neon, llvm_arm_sve, llvm_avx512, math, nvvm, omp, pdl, pdl_interp, quant, rocdl, scf, sdbm, shape, spv, std, tensor, test, tosa, vector
      --convert-linalg-to-std                           -   Convert the operations from the linalg dialect into the Standard dialect
      --convert-scf-to-std                              -   Convert SCF dialect to Standard dialect, replacing structured control flow with a CFG
      --convert-shape-to-std                            -   Convert operations from the shape dialect into the standard dialect
      --convert-std-to-llvm                             -   Convert scalar and vector operations from the Standard to the LLVM dialect
      --convert-std-to-spirv                            -   Convert Standard dialect to SPIR-V dialect
      --legalize-std-for-spirv                          -   Legalize standard ops for SPIR-V lowering
      --std-b

The pass `convert-scf-to-std` seems promising as it intends to convert the `scf` dialect to `std` dialect. 

Let's see if running the `convert-scf-to-std` pass right before the `func-bufferize` pass will get rid of our exception. 

In [8]:
passes = [
    "--test-sparsification=lower",
    "--linalg-bufferize",
    "--convert-scf-to-std", # we added this pass
    "--func-bufferize",
    "--tensor-bufferize",
    "--tensor-constant-bufferize",
    "--finalizing-bufferize",
    "--convert-linalg-to-loops",
    "--convert-scf-to-std",
    "--convert-std-to-llvm",
]
result = cli.apply_passes(mlir_bytes, passes)
print(result[:500])

module attributes {llvm.data_layout = ""}  {
  llvm.func @malloc(i64) -> !llvm.ptr<i8>
  llvm.mlir.global private constant @__constant_8xf32(dense<0.000000e+00> : tensor<8xf32>) : !llvm.array<8 x f32>
  llvm.func @sparseValuesF32(!llvm.ptr<i8>) -> !llvm.struct<(ptr<f32>, ptr<f32>, i64, array<1 x i64>, array<1 x i64>)> attributes {sym_visibility = "private"}
  llvm.func @sparseIndices64(!llvm.ptr<i8>, i64) -> !llvm.struct<(ptr<i64>, ptr<i64>, i64, array<1 x i64>, array<1 x i64>)> attributes {sym_


This fixes our issue, but it's not exactly clear that this is the best solution. 

Since phase ordering is an open problem in compilers research, we can't be sure that where we put ran pass in the correct place.

We know that immediately before the `func-bufferize` pass is the latest place we can run the `convert-scf-to-std` pass and maintain correctness.

It would be nice to know the earliest place we can run the `convert-scf-to-std` pass. Let's find out where `scf.condition` operation came from. 

If we look at the information shown by `cli.debug_passes(mlir_bytes, passes)` earlier, we'll see the inputs to all the passes. We will also see that the first occurrence of `scf.condition` came as an input to the `linalg-bufferize` pass. This means that `scf.condition` was generated by the pass that was run right before the `linalg-bufferize` pass, i.e. the `test-sparsification=lower` pass. This means that the earliest place we can put the `convert-scf-to-std` pass is after the `test-sparsification=lower` pass.

Thus, there are two options for the orderings of our passes. 

In [9]:
pass_list_1 = [
    "--test-sparsification=lower",
    "--convert-scf-to-std", # comes second in the list
    "--linalg-bufferize",
    "--func-bufferize",
    "--tensor-bufferize",
    "--tensor-constant-bufferize",
    "--finalizing-bufferize",
    "--convert-linalg-to-loops",
    "--convert-scf-to-std",
    "--convert-std-to-llvm",
]
pass_list_2 = [
    "--test-sparsification=lower",
    "--linalg-bufferize",
    "--convert-scf-to-std", # comes third in the list
    "--func-bufferize",
    "--tensor-bufferize",
    "--tensor-constant-bufferize",
    "--finalizing-bufferize",
    "--convert-linalg-to-loops",
    "--convert-scf-to-std",
    "--convert-std-to-llvm",
]

Let's get the final lowered code of both. 

In [10]:
result_1 = cli.apply_passes(mlir_bytes, pass_list_1)
result_2 = cli.apply_passes(mlir_bytes, pass_list_2)

Let's see if they are any different. 

In [11]:
result_1 == result_2

True

It seems that both pass orderings create the same lowered code, so (at least for the input MLIR code used in this example) whether we put the `convert-scf-to-std` second or third in the list doesn't matter. 